In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sklearn
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('Train_seers_accuracy/Train_seers_accuracy.csv')

## Missing Value Imputation

In [4]:
df.Payment_Mode.fillna('Credit/Debit Card',inplace=True)

## Feature Engineering

In [5]:
#Feature Engineering
#Converting DOB and Transaction Date to Pandas Datetime format & creating new variables

import datetime as dt
now = dt.datetime.now()

df['Transaction_Date'] = df['Transaction_Date'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%d-%b-%y'))

df['DOB'] = df['DOB'].apply(lambda x: 
                                    dt.datetime.strptime(x,'%d-%b-%y'))

df['DOB'] = df['DOB'].where(df['DOB'] < now, df['DOB'] -  np.timedelta64(100, 'Y'))

df['Age'] = (now - df.DOB).astype('<m8[Y]') 

df['Transaction_Year'] = df.Transaction_Date.apply(lambda x: x.year)
df['Transaction_Month'] = df.Transaction_Date.apply(lambda x: x.month)
df['Transaction_Day'] = df.Transaction_Date.apply(lambda x: x.day)

df['Visits'] = df.groupby('Client_ID')['Client_ID'].transform('count')


In [6]:
#Creating Year wise Data frame subsets
df_2003 = df[df.Transaction_Year==2003]
df_2004 = df[df.Transaction_Year==2004]
df_2005 = df[df.Transaction_Year==2005]
df_2006 = df[df.Transaction_Year==2006]

In [7]:
#Creating variables for each year of purchase and flag it to 1 or 0 based on purchase Made

list_2003 = list(df[df.Transaction_Year==2003]['Client_ID'])
list_2004 = list(df[df.Transaction_Year==2004]['Client_ID'])
list_2005 = list(df[df.Transaction_Year==2005]['Client_ID'])
list_2006 = list(df[df.Transaction_Year==2006]['Client_ID'])

df['Purchase_2003'] = df.Client_ID.isin(list_2003).map({True:1,False:0})
df['Purchase_2004'] = df.Client_ID.isin(list_2004).map({True:1,False:0})
df['Purchase_2005'] = df.Client_ID.isin(list_2005).map({True:1,False:0})
df['Purchase_2006'] = df.Client_ID.isin(list_2006).map({True:1,False:0})

In [8]:
df['Purchase_Total'] = df.ix[:,-4:].sum(axis=1)

In [9]:
def getCountVar(compute_df, count_df, var_name):
    grouped_df = count_df.groupby(var_name)
    count_dict = {}
    for name, group in grouped_df:
        count_dict[name] = group.shape[0]

    count_list = []
    for index, row in compute_df.iterrows():
        name = row[var_name]
        count_list.append(count_dict.get(name, 0))
    return count_list

In [10]:
df.columns

Index([u'Transaction_ID', u'Transaction_Date', u'Store_ID', u'Number_of_EMI',
       u'Purchased_in_Sale', u'Var1', u'Var2', u'Var3', u'Client_ID',
       u'Gender', u'DOB', u'Referred_Friend', u'Sales_Executive_ID',
       u'Sales_Executive_Category', u'Lead_Source_Category', u'Payment_Mode',
       u'Product_Category', u'Transaction_Amount', u'Age', u'Transaction_Year',
       u'Transaction_Month', u'Transaction_Day', u'Visits', u'Purchase_2003',
       u'Purchase_2004', u'Purchase_2005', u'Purchase_2006',
       u'Purchase_Total'],
      dtype='object')

In [11]:
df["Store_Count"] = getCountVar(df, df, "Store_ID")
df["Purchased_in_Sale_Count"] = getCountVar(df, df, "Purchased_in_Sale")
df["Gender_Count"] = getCountVar(df, df, "Gender")
df["Referred_Friend_Count"] = getCountVar(df, df, "Referred_Friend")
df["Sales_Executive_Category_Count"] = getCountVar(df, df, "Sales_Executive_Category")
df["Lead_Source_Category_Count"] = getCountVar(df, df, "Lead_Source_Category")
df["Payment_Mode_Count"] = getCountVar(df, df, "Payment_Mode")
df["Product_Category_Count"] = getCountVar(df, df, "Product_Category")

In [13]:
df1 = df.copy()

In [14]:
df['Cross_Sell'] = 0

In [ ]:
#Visits Per Year Calculation

In [15]:
#Target Variable Creation
#df.ix[(df.Purchase_2003==1) & (df.Purchase_2004==1),'Cross_Sell'] = 1
#df.ix[(df.Purchase_2004==1) & (df.Purchase_2005==1),'Cross_Sell'] = 1
#df.ix[(df.Purchase_2005==1) & (df.Purchase_2006==1),'Cross_Sell'] = 1

df.Cross_Sell = (df['Visits'] - min(df['Visits'])) / (max(df['Visits']) - min(df['Visits']))

#df.ix[df.Visits>1,'Cross_Sell'] = 1

In [ ]:
#df_2003['Visits'] = df_2003.groupby('Client_ID')['Client_ID'].transform('count')
#df_2004['Visits'] = df_2004.groupby('Client_ID')['Client_ID'].transform('count')
#df_2005['Visits'] = df_2005.groupby('Client_ID')['Client_ID'].transform('count')
#df_2006['Visits'] = df_2006.groupby('Client_ID')['Client_ID'].transform('count')

In [ ]:
#df_2003['Cross_Sell'] = (df_2003['Visits'] - min(df_2003['Visits'])) / (max(df_2003['Visits']) - min(df_2003['Visits']))
#df_2004['Cross_Sell'] = (df_2004['Visits'] - min(df_2004['Visits'])) / (max(df_2004['Visits']) - min(df_2004['Visits']))
#df_2005['Cross_Sell'] = (df_2005['Visits'] - min(df_2005['Visits'])) / (max(df_2005['Visits']) - min(df_2005['Visits']))
#df_2006['Cross_Sell'] = (df_2006['Visits'] - min(df_2006['Visits'])) / (max(df_2006['Visits']) - min(df_2006['Visits']))

## EDA

In [16]:
df.head()

,Transaction_ID,Transaction_Date,Store_ID,Number_of_EMI,Purchased_in_Sale,Var1,Var2,Var3,Client_ID,Gender,...,Purchase_Total,Store_Count,Purchased_in_Sale_Count,Gender_Count,Referred_Friend_Count,Sales_Executive_Category_Count,Lead_Source_Category_Count,Payment_Mode_Count,Product_Category_Count,Cross_Sell
0,TRA98825550,2003-01-01,STO1281,2,N,1,1,1,345821599,F,...,1,4960,418293,117039,383739,221641,241935,304554,84575,0
1,TRA98825710,2003-01-01,STO1247,2,N,1,2,1,345821734,M,...,1,37790,418293,322499,383739,221641,241935,304554,84575,0
2,TRA98823874,2003-01-01,STO1244,2,N,1,1,1,345820365,F,...,1,30984,418293,117039,383739,221641,241935,304554,84575,0
3,TRA98823889,2003-01-01,STO1256,2,N,1,1,1,345820377,M,...,1,2579,418293,322499,383739,221641,241935,304554,84575,0
4,TRA98824521,2003-01-01,STO1445,2,N,1,2,1,345820841,F,...,1,123,418293,117039,383739,221641,18154,61433,84575,0


In [37]:
grp = df.groupby('Client_ID')

days_dict = {}

for name, group in grp:
        days_dict[name] = (max(np.array(group['Transaction_Date'])) - min(np.array(group['Transaction_Date']))).astype('timedelta64[D]') / np.timedelta64(1, 'D')
        
days_list = []
for index, row in df.iterrows():
        name = row['Client_ID']
        days_list.append(days_dict.get(name, 0))
        


In [45]:
df['days_bt_purchase'] = days_list

In [47]:
df.days_bt_purchase.replace(inplace=True,to_replace=0,value=1600)

## Model Building

In [ ]:
df.columns

In [ ]:
#Important Features :- Transaction Amount , Purchase , Age , Purchase Total , Purchase in Sale , Var 1 , Var 3 ,Product Category

In [ ]:
sorted(zip(rf.feature_importances_,X_df.columns),reverse=True)

In [49]:
from sklearn.preprocessing import LabelEncoder

features = ['Client_ID','Cross_Sell']

var_mod = ['Purchased_in_Sale','Var1','Var2','Var3','Gender','Referred_Friend','Sales_Executive_Category',\
           'Lead_Source_Category','Payment_Mode','Product_Category'
           ]

var_to_drop = ['Transaction_ID','Transaction_Date','DOB','Store_ID','Sales_Executive_ID','Purchase_Total',\
              'Purchased_in_Sale_Count','Purchased_in_Sale','Lead_Source_Category_Count','Var3','Transaction_Year',\
               'Payment_Mode_Count','Sales_Executive_Category','Var1','Referred_Friend','Sales_Executive_Category_Count',\
               'Referred_Friend_Count']



#for i in var_mod:
#    dummy = pd.get_dummies(df[i],prefix=i)
#    df.drop([i],axis=1,inplace=True)
#    df = pd.concat([df,dummy],axis=1)

cols = [col for col in df.columns if col not in features]

X_df = df[cols]
y_df = df.Cross_Sell

for var in var_mod:
    lb = LabelEncoder()
    full_var_data = X_df[var].astype('str')
    temp = lb.fit_transform(np.array(full_var_data))
    X_df[var] = lb.transform(np.array( X_df[var] ).astype('str'))   
    
X_df.drop(var_to_drop,inplace=True,axis=1)


In [50]:
X_df.head()

,Number_of_EMI,Var2,Gender,Lead_Source_Category,Payment_Mode,Product_Category,Transaction_Amount,Age,Transaction_Month,Transaction_Day,Visits,Purchase_2003,Purchase_2004,Purchase_2005,Purchase_2006,Store_Count,Gender_Count,Product_Category_Count,days_bt_purchase
0,2,0,2,0,2,0,17455,44,1,1,1,1,0,0,0,4960,117039,84575,1600
1,2,1,3,0,2,0,16503,39,1,1,1,1,0,0,0,37790,322499,84575,1600
2,2,0,2,0,2,0,15012,57,1,1,1,1,0,0,0,30984,117039,84575,1600
3,2,0,3,0,2,0,16051,55,1,1,1,1,0,0,0,2579,322499,84575,1600
4,2,1,2,2,1,0,15108,54,1,1,1,1,0,0,0,123,117039,84575,1600


In [51]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
import numpy as np
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.grid_search import GridSearchCV

rf = RandomForestRegressor(n_estimators=100,n_jobs=-1)

k_range = range(1,100,10)

param_grid = dict(n_estimators=k_range)

grid = GridSearchCV(rf,param_grid,cv=5,scoring='mean_squared_error')

grid.fit(X_df,y_df)

print grid.grid_scores_

import matplotlib.pyplot as plt
%matplotlib inline

grid_mean_scores = [result.mean_validation_score for result in grid.grid_scores_]
plt.plot(k_range,grid_mean_scores)

print grid.best_score_
print grid.best_params_
print grid.best_estimator_



In [52]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=500,n_jobs=-1,max_depth=10)

In [ ]:
rf.fit(X_df,y_df)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [ ]:
print rf.score(X_df,y_df)

In [ ]:
y_pred = rf.predict(X_df)

In [ ]:
df['Cross_Sell'] = y_pred

## Output Creation

In [ ]:
df_out = df[['Client_ID','Cross_Sell']].drop_duplicates()
df_out = df_out.drop_duplicates(cols='Client_ID', take_last=True)
df_out.to_csv("new_model.csv", index=False)

In [ ]:
len(df_out)

417107

# Baseline Naive Bayes

In [ ]:
#data = pd.read_csv("../datasets/av_seers_accuracy/Train_seers_accuracy/Train_seers_accuracy.csv", usecols=['Client_ID'])

In [ ]:
#client_ids = data.Client_ID.value_counts().reset_index().rename(columns={"Client_ID": "Freq", "index":"Client_ID"})

In [ ]:
#client_ids['Cross_Sell'] = (client_ids['Freq'] - min(client_ids['Freq'])) / (max(client_ids['Freq']) - min(client_ids['Freq']))

In [ ]:
#[['Client_ID', 'Cross_Sell']].to_csv("../datasets/av_seers_accuracy/baseline2.csv", index=False)

In [ ]:
2 + 2